In [35]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os

def load_json(file_path):
    """Load and return a JSON object from a file."""
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: File not found -> {file_path}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file_path}: {e}")
    return None


def update_json(data, key, value): # Modify/Add Data
    data[key] = value
    return data

def save_json(file_path, data): # Save JSON Data
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [61]:
# creates a chart with the interpretation selections of Chat GPT, Llama and the Rwp minimization
#includes synthesis, interpretation info, balance score, Rwp, and likelihoods for each LLM

def find_all_choices(json_file, sample):
    sample_choice = {}
    if 1 == 1:
        llama_probs = []
        chat_probs = []
        rwp_vals = []
        sample_choice[sample] = {}
        for interpret, info in json_file[sample].items():
            if interpret == "Synth_Conditions":
                continue
            try:
                llama_score = info["unnormalized_posterior_llama"]
                chat_score = info["unnormalized_posterior"]
                rwp = info["rwp"]
                if all(isinstance(x, (int, float)) for x in [llama_score, chat_score, rwp]):
                    llama_probs.append((interpret, llama_score))
                    chat_probs.append((interpret, chat_score))
                    rwp_vals.append((interpret, rwp))
            except KeyError as e:
                print(f"Missing key in sample '{sample}', interpretation '{interpret}': {e}")
        
        sample_choice[sample]["llama_choice"] = max(llama_probs, key=lambda x: x[1])[0] if llama_probs else None
        sample_choice[sample]["chat_choice"] = max(chat_probs, key=lambda x: x[1])[0] if chat_probs else None
        sample_choice[sample]["rwp_choice"] = min(rwp_vals, key=lambda x: x[1])[0] if rwp_vals else None
    
    return sample_choice
file_path = "/Users/admin/Documents/GitHub/LLM_XRD_Interpretation_Validation/Data/prompt3/interpretations_llm_v1_llama1.json"
json_file = load_json(file_path)
find_all_choices(json_file, "ARR_39")

{'ARR_39': {'llama_choice': 'I_2', 'chat_choice': 'I_1', 'rwp_choice': 'I_1'}}

In [65]:
import pandas as pd
file_path = "/Users/admin/Documents/GitHub/LLM_XRD_Interpretation_Validation/Data/prompt3/interpretations_llm_v1_llama1.json"
json_file = load_json(file_path)
#sample_id = 'ARR_39'  # sample name

def format_predicted_phases(json_file, sample_id, chat_choice): 
    # Format the predicted phases with the weight fraction to be used in a table
    phases = json_file[sample_id][chat_choice]["phases"]
    wfs = json_file[sample_id][chat_choice]["weight_fraction"]
    wfs = [round(wf, 3) for wf in wfs]
    phase_list = ""
    for i in range(len(phases)):
        if not phase_list:
            phase_list += f"({phases[i]} {wfs[i]}%)"
        else: phase_list += f", ({phases[i]} {wfs[i]}%)"
    return phase_list

# Helper function to extract values safely
def assemble_df(sample_id, sample_choice, json_file):
    rows = []
    # assign Chat GPT row values
    chat_row = {}
    chat_choice = sample_choice[sample_id]["chat_choice"]
    chat_row["Selection Method"] = "Chat GPT"
    chat_row["Choice"] = chat_choice
    chat_row["Predicted Phases"] = format_predicted_phases(json_file, sample_id, chat_choice)
    chat_row["Balance Score"] = json_file[sample_id][chat_choice]["balance_score"]
    chat_row["Rwp"] = json_file[sample_id][chat_choice]["rwp"]
    chat_row["Chat GPT Likelihood"] = json_file[sample_id][chat_choice]["LLM_interpretation_likelihood"]
    chat_row["Llama Likelihood"] = json_file[sample_id][chat_choice]["LLM_interpretation_likelihood_llama"]

    #assign Llama row values
    llama_row = {}
    llama_choice = sample_choice[sample_id]["llama_choice"]
    llama_row["Selection Method"] = "Llama"
    llama_row["Choice"] = llama_choice
    llama_row["Predicted Phases"] = format_predicted_phases(json_file, sample_id, llama_choice)
    llama_row["Balance Score"] = json_file[sample_id][llama_choice]["balance_score"]
    llama_row["Rwp"] = json_file[sample_id][llama_choice]["rwp"]
    llama_row["Chat GPT Likelihood"] = json_file[sample_id][llama_choice]["LLM_interpretation_likelihood"]
    llama_row["Llama Likelihood"] = json_file[sample_id][llama_choice]["LLM_interpretation_likelihood_llama"]

    #assign lowest Rwp values
    rwp_row = {}
    rwp_choice = sample_choice[sample_id]["rwp_choice"]
    rwp_row["Selection Method"] = "Lowest Rwp"
    rwp_row["Choice"] = rwp_choice
    rwp_row["Predicted Phases"] = format_predicted_phases(json_file, sample_id, rwp_choice)
    rwp_row["Balance Score"] = json_file[sample_id][rwp_choice]["balance_score"]
    rwp_row["Rwp"] = json_file[sample_id][rwp_choice]["rwp"]
    rwp_row["Chat GPT Likelihood"] = json_file[sample_id][rwp_choice]["LLM_interpretation_likelihood"]
    rwp_row["Llama Likelihood"] = json_file[sample_id][rwp_choice]["LLM_interpretation_likelihood_llama"]
    rows = [chat_row, llama_row, rwp_row]
    return rows

for sample_id in json_file.keys(): # go through every sample and display a chart with relevant information 
    sample_choice = find_all_choices(json_file, sample_id)
    rows = assemble_df(sample_id, sample_choice, json_file)
    pd.set_option('display.max_colwidth', None)
    df = "df" + sample_id
    df = pd.DataFrame(rows)
    df = df[["Selection Method", "Choice", "Predicted Phases", "Balance Score", "Rwp", "Chat GPT Likelihood", "Llama Likelihood"]]
    print(sample_id)
    display(df)


PG_1048_1


,Selection Method,Choice,Predicted Phases,Balance Score,Rwp,Chat GPT Likelihood,Llama Likelihood
0,Chat GPT,I_3,"(V2O5_59 51.188%), (V3Cr7O15_167 48.812%), (Cr2O3_167 0.0%)",0.9744,4.74,0.70,0.8
1,Llama,I_3,"(V2O5_59 51.188%), (V3Cr7O15_167 48.812%), (Cr2O3_167 0.0%)",0.9744,4.74,0.70,0.8
2,Lowest Rwp,I_1,"(V20CrO52_59 4.341%), (V3Cr7O15_167 46.84%), (Cr2O3_167 0.0%), (V2O5_59 48.82%)",0.9711,4.60,0.55,0.6


ARR_39


,Selection Method,Choice,Predicted Phases,Balance Score,Rwp,Chat GPT Likelihood,Llama Likelihood
0,Chat GPT,I_1,(Ga33O50_12 100.0%),0.375,20.11,0.35,0.4
1,Llama,I_2,(Ga2O3_12 100.0%),0.375,20.14,0.35,0.6
2,Lowest Rwp,I_1,(Ga33O50_12 100.0%),0.375,20.11,0.35,0.4


TRI_62


...

,Selection Method,Choice,Predicted Phases,Balance Score,Rwp,Chat GPT Likelihood,Llama Likelihood
0,Chat GPT,I_5,"(CaC2_2 29.148%), (Ca5V3O13_176 21.802%), (V9O17_2 18.774%), (CaCO3_167 15.794%), (V2O5_15 14.482%)",0.9919,18.54,0.75,0.65
1,Llama,I_5,"(CaC2_2 29.148%), (Ca5V3O13_176 21.802%), (V9O17_2 18.774%), (CaCO3_167 15.794%), (V2O5_15 14.482%)",0.9919,18.54,0.75,0.65
2,Lowest Rwp,I_5,"(CaC2_2 29.148%), (Ca5V3O13_176 21.802%), (V9O17_2 18.774%), (CaCO3_167 15.794%), (V2O5_15 14.482%)",0.9919,18.54,0.75,0.65


ARR_46


,Selection Method,Choice,Predicted Phases,Balance Score,Rwp,Chat GPT Likelihood,Llama Likelihood
0,Chat GPT,I_3,"(Ti12O25_61 65.616%), (CaTi4(PO4)6_148 34.384%)",0.70930,7.22,0.60,0.65
1,Llama,I_3,"(Ti12O25_61 65.616%), (CaTi4(PO4)6_148 34.384%)",0.70930,7.22,0.60,0.65
2,Lowest Rwp,I_2,"(CaP2(HO2)4_2 70.209%), (Ti12O25_61 15.823%), (CaTi4(PO4)6_148 13.969%)",0.79164,6.91,0.35,0.25


TRI_105


,Selection Method,Choice,Predicted Phases,Balance Score,Rwp,Chat GPT Likelihood,Llama Likelihood
0,Chat GPT,I_1,"(V2O3_167 69.966%), (CaO_225 17.014%), (Ca5V3O13_176 13.02%)",0.993056,5.72,0.65,0.7
1,Llama,I_1,"(V2O3_167 69.966%), (CaO_225 17.014%), (Ca5V3O13_176 13.02%)",0.993056,5.72,0.65,0.7
2,Lowest Rwp,I_1,"(V2O3_167 69.966%), (CaO_225 17.014%), (Ca5V3O13_176 13.02%)",0.993056,5.72,0.65,0.7


In [80]:
import textwrap
file_path = "/Users/admin/Documents/GitHub/LLM_XRD_Interpretation_Validation/Data/prompt3/interpretations_llm_v1_llama1.json"
json_file = load_json(file_path)
def print_synthconditions(sample_id,json_file):
    synth_conditions = json_file[sample_id]["Synth_Conditions"]
    target = synth_conditions["Target"]
    precursor1 = synth_conditions["Precursor 1"]
    precursor2 = synth_conditions["Precursor 2"]
    if "Precursor 3" in synth_conditions:
        precursor3 = synth_conditions["Precursor 3"]
    furnace = synth_conditions["Furnace"]
    tempc = synth_conditions["Temperature (C)"]
    tempk = synth_conditions["Temperature (K)"]
    dwell = synth_conditions["Dwell Duration (h)"]
    if "Precursor 3" in synth_conditions:
        print_conditions =textwrap.dedent(f"""
        Target: {target}
        Precursors: {precursor1}, {precursor2}, {precursor3}
        Furnace: {furnace}
        Temperature (C): {tempc}
        Temperature (K): {tempk}
        Dwell Duration (h): {dwell}
        """)
    else:
        print_conditions = textwrap.dedent(f"""
        Synthesis Conditions:                                   
        Target: {target}
        Precursors: {precursor1}, {precursor2}
        Furnace: {furnace}
        Temperature (C): {tempc}
        Temperature (K): {tempk}
        Dwell Duration (h): {dwell}
        """)
    return print_conditions
sample_id = "TRI_183"
print_conditions = print_synthconditions(sample_id,json_file)
print(print_conditions)


Synthesis Conditions:                                   
Target: MgVO3
Precursors: MgCO3, V2O5
Furnace: BF
Temperature (C): 500.0
Temperature (K): 773.15
Dwell Duration (h): 12.0



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

def find_choices(sample, json_file, model):
    rwp_vals = []
    model_scores = []
    for interpret, info in json_file[sample].items():
        if interpret == "Synth_Conditions":
            continue
        rwp = info["rwp"]
        if model == "Llama":
            model_score = info["unnormalized_posterior_llama"]
        if model == "Chat GPT":
            model_score = info["unnormalized_posterior"]
        rwp_vals.append((interpret, rwp))
        model_scores.append((interpret, model_score))
    sample_choice_rwp = min(rwp_vals, key=lambda x: x[1])[0] if rwp_vals else None
    sample_choice_model = max(model_scores, key=lambda x: x[1])[0] if rwp_vals else None
    return [sample_choice_rwp, sample_choice_model]

def graph_interpretation_likelihoods(json_file, sample_id, model):
    # Interpretation names and probabilities
    interpretations = list(json_file[sample_id].keys())
    interpretations = [i for i in interpretations if i != "Synth_Conditions"]
    probabilities = []
    phase_fractions = []
    phase_labels = []
    label_colors = []
    for interpret in interpretations:
        if model == "Llama": #collect llama probabilities 
            probabilities.append(json_file[sample_id][interpret]["posterior_probability_llama"]) 
        if model == "Chat GPT": #collect chat gpt probabilities
            probabilities.append(json_file[sample_id][interpret]["posterior_probability"])
        phase_fractions.append(json_file[sample_id][interpret]["weight_fraction"])
        phase_labels.append(json_file[sample_id][interpret]["phases"])

    # assign label colors based on if the LLM or Rwp minimization favors the Interpretation
    choices = find_choices(sample_id,json_file,model)
    for interpret in interpretations:    
        if interpret == choices[0]:
            label_colors.append("blue")
        if interpret == choices[1]:
            label_colors.append("royalblue")
        else: label_colors.append("black")

    phase_colors = ['#f87060', '#75aadb', '#b3e34f', '#ba79c6']
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(10, 4))  # Wider for more space

    # Plot each interpretation as stacked horizontal bar
    for i in range(len(interpretations)):
        left = 0
        for j in range(len(phase_fractions[i])):
            width = phase_fractions[i][j]
            if width > 0:
                ax.barh(i, width, left=left, color=phase_colors[j], height=0.6)
                left += width

    # Add interpretation names inside the bar (or just left of the bar)
    for i, (name, prob, color) in enumerate(zip(interpretations, probabilities, label_colors)):
        ax.text(-1, i, name, va='center', ha='right', fontweight='bold', fontsize=11, color=color)
        ax.text(prob + 1.5, i, f"{prob}%", va='center', ha='left', fontsize=10)

    # Customize legend
    legend_handles = [Patch(color=phase_colors[i], label=phase_labels[i]) for i in range(len(phase_colors))]
    ax.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1.02, 0.5), borderaxespad=1)

    # Axis and layout
    ax.set_xlim(0, max(probabilities) + 10)
    ax.set_yticks([])  # Remove y-axis ticks
    ax.set_xlabel('Interpretation Probability (%)', fontsize=12, fontweight='bold')
    ax.grid(axis='x', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

sample_id = "TRI_84"
model = "Llama"
graph_interpretation_likelihoods(json_file, sample_id, model)


UnboundLocalError: cannot access local variable 'find_choices' where it is not associated with a value